In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import re
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

%load_ext autoreload
%autoreload 2

sys.path.append('../')
import affordability as af



In [ ]:
def car_value(purchase_price, num_years):
    """
    Depreciation at 40% for first year and 20%/year thereafter.
    Taken from the Dog and Lemon guide at https://dogandlemon.com/articles/depreciation-new-zealand.
    """
    x = purchase_price
    n = num_years
    k = math.floor(n)
    kk = n - k
    print(k, kk)
    if n <= 0:
        v = x
    else:        
        v = 0.80**(max(0, n-1))*0.6*x
    return v

car_value(73000, 14.2)

# Aggregate regional rents to rental areas and write to JSON

In [2]:
# Slice rental data from the following date to latest date

def jsonize_rents(region, date):
    """
    Given a region (string) and a date (YYYY-MM-DD string),
    read the rents for the given region, slice them from the given date
    to the latest date in the date, aggregate the rents by rental area and number of bedrooms
    ('1', '2', '3', or '4'), and return the result as a dictionary of the form
    rental area -> #bedrooms -> rent geometric mean.
    Some of the mean rents could be ``None``.
    """
    rents = af.get_data(region, 'rents_csv')
    f = af.aggregate_rents(rents, date)
    
    # Drop 5+ bedrooms and round to nearest dollar
    f = f[f['#bedrooms'] != '5+'].copy().round()
    
    # Replace NaN with None to make JSON-compatible 
    f = af.nan_to_none(f)
    
    # Save to dictionary of form rental area -> #bedrooms -> rent geo mean
    d = {area: dict(g[['#bedrooms', 'rent_geo_mean']].values)
      for area, g in f.groupby('rental_area')}

    return d


date = '2016-09-01'

for region in af.REGIONS:
    d = jsonize_rents(region, date)
    path = af.get_path(region, 'rents_json')   
    with path.open('w') as tgt:
        json.dump(d, tgt)

    

# Compute roundtrip commute cost matrix and save to JSON matrix

In [7]:
def build_commute_costs(region):
    """
    Consolidate the data in the commute CSV files for this region and 
    save it into one JSON master file of daily commute cost and time. 
    More specifically, save the dictionary 
    ``{'index_by_name': index_by_name, 'matrix': M}``, where 
    ``index_by_name`` is a dictionary with structure
    area unit name -> row/column index in the lower-triangular half-matrix 
    ``M``, where ``M`` is encoded by a dictionary with structure
    mode -> list of lists of cost-time pairs 
    such that ``M[mode][i][j]`` equals the cost in dollars
    and the time in hours that it takes to travel round-trip by the 
    given mode (specified in the list ``MODES``)
    from the centroid of the area unit with index ``i >= 0`` 
    to the centroid of the area unit with index ``j <= i``.
    """
    # Get rental area names
    rents = af.get_data(region, 'rents_csv')
    names = sorted(rents['rental_area'].unique())
    index_by_name = {name: i for (i, name) in enumerate(names)}
    n = len(names)

    # Create a commute matrix M with costs and durations
    M = {mode: [[(None, None) for j in range(n)] for i in range(n)] 
      for mode in af.MODES}
    frames = []
    for mode in af.MODES:
        f = af.get_data(region, 'commutes_' + mode)
        f['mode'] = mode
        f['orig_index'] = f['orig_name'].map(index_by_name)
        f['dest_index'] = f['dest_name'].map(index_by_name)
        
        # Convert from meters to kilometers and seconds to hours
        f['distance'] /= 1000
        f['duration'] /= 3600

        # Compute costs
        if mode == 'transit':
            # Use separate cost table
            costs = af.get_data(region, 'transit_costs').rename(
              columns={'card_fare': 'cost'})
            f = f.merge(costs)
            # Nullify costs with missing distances
            cond = f['distance'].isnull()
            f.loc[cond, 'cost'] = np.nan
        else:
            # Multiply distance by cost per distance
            f['cost'] = af.COST_BY_MODE[mode]*f['distance']

        frames.append(f[['mode', 'orig_index', 'dest_index', 'cost', 'duration']].copy())

    f = pd.concat(frames)
    return f

    # Use f to create a lower-triangular half-matrix M of roundtrip commute costs and durations
    f = af.nan_to_none(f)
    M = {mode: [[(None, None) for j in range(i + 1)] for i in range(n)] 
      for mode in af.MODES}
    for mode, oi, di, cost, dur in f.itertuples(index=False):
        if 
    for mode in ['walking', 'bicyling', 'driving']:
        for i in range(n):
            for j in range(i + 1):
                try:
                    distance = M[mode][i][j][0] + M[mode][j][i][0]
                    duration = M[mode][i][j][1] + M[mode][j][i][1]
                    MM[mode][i][j] = (
                      round(af.COST_BY_MODE[mode]*distance, 2),
                      round(duration, 2)
                      )
                except TypeError:
                    # Defaults to MM[mode][i][j] = (None, None) 
                    pass  
    
    
    data = {'index_by_name': index_by_name, 'matrix': MM}
    return data

In [11]:
f = build_commute_costs('wellington')

In [14]:
cond1 = f['orig_index'] < f['dest_index']
cond2 = f['orig_index'] > f['dest_index']
f1 = f[cond1].copy().sort_values(['mode', 'orig_index', 'dest_index'])
f2 = f[cond2].copy().sort_values(['mode', 'orig_index', 'dest_index'], 
  ascending=[True, False, False])
print(f1.head())
print(f2.tail())

          mode  orig_index  dest_index  cost  duration
938  bicycling           0           1   0.0  6.420556
956  bicycling           0           2   0.0  0.824444
958  bicycling           0           3   0.0  1.555833
927  bicycling           0           4   0.0  0.398056
943  bicycling           0           5   0.0  2.200278
         mode  orig_index  dest_index  cost   duration
1439  walking           3           1   0.0  12.970556
1450  walking           3           0   0.0   2.940833
1313  walking           2           1   0.0  15.197500
1324  walking           2           0   0.0   1.382222
483   walking           1           0   0.0  15.645000
